# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

### Name : Udit Desai

### Roll no. 17CS30044

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

In [72]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [73]:
#Load the IMDB dataset. You can load it using pandas as dataframe
df = pd.read_csv('/home/udit/Downloads/IMDB Dataset.csv')
print(df)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [74]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

lm = WordNetLemmatizer()
ps = PorterStemmer()

stopword = set(stopwords.words('english'))
rows = df.shape[0]
processed_reviews = []
for i in range(rows):
    rev = df.iloc[i][0]
    
    #convert to lower case
    rev = rev.lower()    
   
    #remove html tags
    exp_html = re.compile(r'<.*?>')
    rev = exp_html.sub('', rev)
    
    #remove URLS
    rev = re.sub(r'^https?:\/\/.*[\r\n]*', '', rev, flags=re.MULTILINE)
    
    #remove non-alphanumeric
    rev = re.sub(r'[^a-zA-Z0-9]', " ", rev)
    
    #remove stopwords
    word_tokens = word_tokenize(rev)
    filtered_sentence = [w for w in word_tokens if not w in stopword]
    rev = " ".join(filtered_sentence)
    
    #lemmatize and stemming
    word_tokens = word_tokenize(rev)
    sentence = [lm.lemmatize(w,"v") for w in word_tokens]
    #sentence = [ps.stem(w) for w in sentence]
    rev = " ".join(sentence)
    processed_reviews.append(rev)
    
    #df.iloc[i][0] = rev
    #print(i)



In [75]:
#print(processed_reviews)

In [76]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
sum = 0
p=0
n=0
rows = len(processed_reviews)
for i in range(rows):
    sum = sum + len(processed_reviews[i].split())
    if df.iloc[i][1]=='positive' :
        p+=1
    else :
        n+=1   
avg =  sum/rows



print("Average length of review is: ", avg)
print("Proportion of positive reviews: ", p/rows)
print("Proportion of negative reviews: ", n/rows)  


Average length of review is:  119.57722
Proportion of positive reviews:  0.5
Proportion of negative reviews:  0.5


# Naive Bayes classifier

In [77]:
# get reviews column from df
reviews = processed_reviews

# get labels column from df
temp = df['sentiment'].to_list()
labels = temp

In [98]:
# Use label encoder to encode labels. Convert to 0/1
print("1 is assumed positive, and 0 is assumed negative")
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
print("Labels : ", end = ' ')
print(encoded_labels)
# print(enc.classes_)

1 is assumed positive, and 0 is assumed negative
Labels :  [1 1 1 ... 0 0 0]


In [79]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.

# train_sentences, test_sentences, train_labels, test_labels

X_train, X_test, y_train, y_test = train_test_split(reviews, encoded_labels, test_size=0.2, random_state=None, stratify = encoded_labels)

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [32]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words
'''
max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(Sentence_list)
'''
vectorizer = CountVectorizer(max_features=3000)
x_train = vectorizer.fit_transform(X_train)
x_train_arr = x_train.toarray()

In [33]:
# Use laplace smoothing for words in test set not present in vocab of train set
# Use laplace smoothing for words in test set not present in vocab of train set
train_list = vectorizer.get_feature_names()
vect = CountVectorizer()
x_test = vect.fit_transform(X_test)
test_list = list(vect.get_feature_names())
not_in_v = set(test_list).difference(set(train_list)) 
niv_list = list(not_in_v)
tot_pos = 0
tot_neg = 0
total_words = len(not_in_v)+3000
pos = np.zeros((total_words,1))
neg = np.zeros((total_words,1))
for i in range(3000):
    num_pos = 0
    num_neg = 0
    for j in range(len(X_train)):
        if y_train[j] == 0:
            num_neg += x_train_arr[j][i]
        else:
            num_pos += x_train_arr[j][i]        
    tot_neg += num_neg
    tot_pos += num_pos
    neg[i] = num_neg + 1
    pos[i] = num_pos + 1
    
for i in range(len(niv_list)): 
    neg[i+3000] = 1
    pos[i+3000] = 1


In [34]:
# Build the model. Don't use the model from sklearn
pfinal = np.zeros((total_words,2)) 
print(tot_pos, tot_neg)
for i in range(total_words):
    pfinal[i][0] = neg[i]/(tot_neg + 3000)
    pfinal[i][1] = pos[i]/(tot_pos + 3000)
    
print(pfinal)


1989728 1960152
[[1.47212238e-04 1.13412367e-04]
 [1.61475016e-03 1.82061977e-03]
 [2.01207038e-04 1.79653219e-04]
 ...
 [5.09384908e-07 5.01824634e-07]
 [5.09384908e-07 5.01824634e-07]
 [5.09384908e-07 5.01824634e-07]]


In [35]:
# Test the model on test set and report Accuracy
x_test_arr = x_test.toarray()
y_pred = []

for i in range(len(X_test)): 
    posprob = 0
    negprob = 0
    temp = list(X_test[i].split())
    for j in range(len(temp)):
        if temp[j] in set(train_list):
            k = train_list.index(temp[j])
            posprob += np.log(pfinal[k][1])
            negprob += np.log(pfinal[k][0])
        else:
            posprob += np.log(pfinal[total_words-1][1])
            negprob += np.log(pfinal[total_words-1][0])
    if (posprob > negprob):
        y_pred.append(1)
    else:
        y_pred.append(0)

correct = 0
for i in range(len(y_test)):
    if (y_test[i] == y_pred[i]):
        correct += 1

print("accuracy is: ", correct/(len(y_test)))


accuracy is:  0.841


# *LSTM* based Classifier

Use the above train and test splits.

In [82]:
# Hyperparameters of the model
vocab_size = 3000
oov_tok = '<OOK>'
embedding_dim = 100
max_length =120
padding_type='post'
trunc_type='post'

In [83]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [84]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 120, 100)          300000    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_12 (Dense)             (None, 24)                3096      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [85]:
num_epochs = 5
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 46s 41ms/step - loss: 0.3699 - accuracy: 0.8366 - val_loss: 0.3188 - val_accuracy: 0.8687
Epoch 2/5
1125/1125 [==============================] - 44s 39ms/step - loss: 0.2699 - accuracy: 0.8910 - val_loss: 0.3022 - val_accuracy: 0.8735
Epoch 3/5
1125/1125 [==============================] - 48s 43ms/step - loss: 0.2335 - accuracy: 0.9077 - val_loss: 0.3089 - val_accuracy: 0.8748
Epoch 4/5
1125/1125 [==============================] - 45s 40ms/step - loss: 0.1946 - accuracy: 0.9241 - val_loss: 0.3264 - val_accuracy: 0.8685
Epoch 5/5
1125/1125 [==============================] - 45s 40ms/step - loss: 0.1593 - accuracy: 0.9402 - val_loss: 0.3475 - val_accuracy: 0.8635


In [93]:
# Calculate accuracy on Test data
# Get probabilities
prediction = model.predict(test_padded)
for i in range(len(prediction)):
    if prediction[i] < 0.5:
        prediction[i] = 0
    else:
        prediction[i] = 1

print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86      5000
           1       0.86      0.85      0.85      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



## Get predictions for random examples

In [99]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
my_prediction = model.predict(padded)
print("Probabilities : \n")
print(model.predict(padded))

# Get labels based on probability 1 if p>= 0.5 else 0

for i in range(len(my_prediction)):
    if my_prediction[i] < 0.5:
        my_prediction[i] = 0
    else:
        my_prediction[i] = 1

print()        
print("Final predictions based on the probablities are: \n")
for i in range(len(my_prediction)):
    print(sentence[i], " : " , end = ' ')
    if(my_prediction[i]==0):
        print("Negative review (0)")
    else :
        print("Positive review (1)")





Probabilities : 

[[0.6480608 ]
 [0.15470463]
 [0.15309083]]

Final predictions based on the probablities are: 

The movie was very touching and heart whelming  :  Positive review (1)
I have never seen a terrible movie like this  :  Negative review (0)
the movie plot is terrible but it had good acting  :  Negative review (0)
